In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_scPDSI = pd.read_csv('annual_scPDSI.csv').dropna()
df_pre = pd.read_csv('annual_pre.csv').dropna()
df_temp = pd.read_csv('annual_temp.csv').dropna()
df_tmax = pd.read_csv('annual_Tmax.csv').dropna()
df_tmin = pd.read_csv('annual_Tmin.csv').dropna()
df_dtr = pd.read_csv('annual_dtr.csv').dropna()
df_pet = pd.read_csv('annual_pet.csv').dropna()
df_vpd = pd.read_csv('annual_VPD.csv').dropna()

In [ ]:
df_scPDSI['CV'] = df_scPDSI.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_pre['CV'] = df_pre.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_temp['CV'] = df_temp.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_tmax['CV'] = df_tmax.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_tmin['CV'] = df_tmin.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_dtr['CV'] = df_dtr.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_pet['CV'] = df_pet.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_vpd['CV'] = df_vpd.iloc[:, 1:].apply(lambda row: row.std() / row.mean(), axis=1)
df_scPDSICV = df_scPDSI[['site', 'CV']]
df_preCV = df_pre[['site', 'CV']]
df_tempCV = df_temp[['site', 'CV']]
df_tmaxCV = df_tmax[['site', 'CV']]
df_tminCV = df_tmin[['site', 'CV']]
df_DTRCV = df_dtr[['site', 'CV']]
df_petCV = df_pet[['site', 'CV']]
df_vpdCV = df_vpd[['site', 'CV']]
dfs = [df_preCV, df_tempCV,df_tmaxCV,df_tminCV,df_DTRCV,df_petCV,df_vpdCV]
from functools import reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on='site'), dfs)
df_merged.columns = ['site','Pre_CV','Temp_CV','Tmax_CV','Tmin_CV',
                    'DTR_CV','Pet_CV','VPD_CV']
df_merged.to_csv('CV.csv',index=False)
dfGL = pd.read_excel('XGboostData.xlsx')
df_XGdata = pd.merge(dfGL,df_merged,on='site')
df_XGdata.to_csv('uncleaned_XGdata.csv')
df_gro = pd.read_csv('uncleaned_XGdata.csv')

df_cleaned = df_gro.copy()

humid_df = df_cleaned[df_cleaned['AI_Class'] == 'Humid']
arid_df = df_cleaned[df_cleaned['AI_Class'] == 'Arid']

def filter_confidence_interval(df):
    confidence_interval = df.drop(columns=['site', 'AI_Class']).apply(lambda x: np.percentile(x, [2.5, 50, 97.5]))
    df_filtered = df.drop(columns=['site', 'AI_Class']).apply(lambda x: x[(x >= np.percentile(x, 2.5)) & (x <= np.percentile(x, 97.5))])
    df_filtered = pd.concat([df[['site', 'AI_Class']], df_filtered], axis=1)
    return df_filtered.dropna(), confidence_interval

humid_filtered, humid_confidence_interval = filter_confidence_interval(humid_df)
arid_filtered, arid_confidence_interval = filter_confidence_interval(arid_df)

df_filtered = pd.concat([humid_filtered, arid_filtered], axis=0)

humid_confidence_df = pd.DataFrame(humid_confidence_interval.T)
humid_confidence_df.columns=['2.5%', '50%', '97.5%']
arid_confidence_df = pd.DataFrame(arid_confidence_interval.T)
arid_confidence_df.columns=['2.5%', '50%', '97.5%']

humid_confidence_df['AI_Class'] = 'Humid'
arid_confidence_df['AI_Class'] = 'Arid'

confidence_df = pd.concat([humid_confidence_df, arid_confidence_df], axis=0)

confidence_df.reset_index(inplace=True)
confidence_df.rename(columns={'index': 'Variable'}, inplace=True)
confidence_df.to_csv('Arid_Humid95.csv',index=False)
df_filtered.to_csv('xgboost95.csv',index=False)
dfcs =pd.read_csv('AridGLfixed_effects.csv')
dfcs['CI Lower'] = dfcs['Estimate'] - 1.96 * dfcs['Std. Error']
dfcs['CI Upper'] = dfcs['Estimate'] + 1.96 * dfcs['Std. Error']

plt.figure(figsize=(4, 8))
sns.barplot(x='Estimate', y='Va', data=dfcs, palette='coolwarm', ci=None)

for index, row in dfcs.iterrows():
    plt.plot([row['CI Lower'], row['CI Upper']], [index, index], color='black')

for index, row in dfcs.iterrows():
    if row['Pr(>|t|)'] < 0.001:
        plt.text(row['Estimate'], index, '***', color='red', ha='center', va='center')
    elif row['Pr(>|t|)'] < 0.01:
        plt.text(row['Estimate'], index, '**', color='red', ha='center', va='center')
    elif row['Pr(>|t|)'] < 0.05:
        plt.text(row['Estimate'], index, '*', color='red', ha='center', va='center')

plt.axvline(x=0, color='red', linestyle='--')
plt.xlabel('Estimates')
plt.ylabel('')
plt.xlim(-0.6,0.6)
plt.savefig('Fig5a.jpg', dpi=600, bbox_inches='tight')
plt.show()
dfcs =pd.read_csv('HumidGLfixed_effects.csv',)

dfcs['CI Lower'] = dfcs['Estimate'] - 1.96 * dfcs['Std. Error']
dfcs['CI Upper'] = dfcs['Estimate'] + 1.96 * dfcs['Std. Error']


plt.figure(figsize=(4, 8))
sns.barplot(x='Estimate', y='Va', data=dfcs, palette='coolwarm', ci=None)
for index, row in dfcs.iterrows():
    plt.plot([row['CI Lower'], row['CI Upper']], [index, index], color='black')

for index, row in dfcs.iterrows():
    if row['Pr(>|t|)'] < 0.001:
        plt.text(row['Estimate'], index, '***', color='red', ha='center', va='center')
    elif row['Pr(>|t|)'] < 0.01:
        plt.text(row['Estimate'], index, '**', color='red', ha='center', va='center')
    elif row['Pr(>|t|)'] < 0.05:
        plt.text(row['Estimate'], index, '*', color='red', ha='center', va='center')

plt.axvline(x=0, color='red', linestyle='--')
plt.xlabel('Estimates')
plt.ylabel('')
plt.xlim(-0.6,0.6)
plt.savefig('Fig5b.jpg', dpi=600, bbox_inches='tight')
plt.show()